In [1]:
:opt no-lint

# 인터프리터 맛보기

* PiH2nd 8.7, 16.7, 17

In [2]:
data Expr = Val Int
          | Add Expr Expr
          deriving Show

In [3]:
eval :: Expr -> Int
eval (Val n) = n
eval (Add e1 e2) = eval e1 + eval e2

In [4]:
eval (Add (Add (Val 2) (Val 3)) (Val 4)) -- (2 + 3) + 4

9

In [5]:
eval $ (Val 2 `Add` Val 3) `Add` (Val 4) -- (2 + 3) + 4

9

위에서 살펴본 가장 간단한 인터프리터는 분명 맞는 결과를 계산한다.
하지만 내가 정의한 언어에서 계산 순서를 정하고 싶다면 어떨까?
예를 들어 어떤 언어는 함수를 호출할 때 왼쪽에 있는 인자부터 먼저 계산한다던가
또 다른 언어는 오른쪽에 있는 인자부터 계산한다던가 하는 순서를 정해 놓기도 하고,
또 어떤 언어는 그런 순서는 정의되어 있지 않고 언어를 구현하는 사람이
(즉 인터프리터나 컴파일러 등을 만드는 개발자가) 자유롭게 해도 무방하다고 약속하는 경우도 있다.

만약 우리의 정수 덧셈 언어에서 덧셈의 왼쪽부터 먼저 항상 계산하도록 하려면 eval과는 조금 다른 방식으로 인터프리터를 정의해야 된다.
이제부터는 스택이라는 데이타 구조를 활용해 연산자의 왼쪽부터 계산 순서를 강제하는 (... 것처럼 보이는 ...) 함수를 정의해 보자.

"강제하는 (... 것처럼 보이는 ...)"이라고 이야기하는 이유는 하스켈이 기본적으로 게으른 계산법을 따르는 프로그래밍 언어이기 때문에
아래와 같은 코드에서는 GHC가 제공하는 언어 확장 기능을 사용해야 실제로 계산 순서가 강제되지만 여기서는 그냥 개념적으로만 이해를
돕기 위한 코드로 생각하자. (실제로 하스켈 프로그램 실행시에 우리가 의도한 대로 계산 순서를 강제하려면
`BangPattenrs`라는 GHC 확장을 허용하고 `push !n s = n : s`라고 정의를 수정하면 되긴 하는데 수업 진행상 여기까지 ㅁ로라도 된다)

In [6]:
type Stack = [ Int ]

-- 스택 기본 연산
push :: Int -> Stack -> Stack
push n s = n : s
pop :: Stack -> Stack
pop (n:s) = s
top :: Stack -> Int
top (n:_) = n

-- 스택 기본 연산으로 스택을 활용한 덧셈 연산 정의
addL :: Stack -> Stack
addL s2 = push (n1+n2) s0
  where  -- addL [3,4,5,6] 즉 s2 = [3,4,5,6] 이면
    n2 = top s2  -- n2 = 3
    s1 = pop s2  -- s1 = [4,5,6]
    n1 = top s1  -- n1 = 4
    s0 = pop s1  -- s0 = [5,6]

evalL :: Expr -> Stack -> Stack
evalL (Val n)     s = push n s
evalL (Add e1 e2) s = addL s2
  where 
  s1 = evalL e1 s
  s2 = evalL e2 s1

In [7]:
evalL (Add (Add (Val 2) (Val 3)) (Val 4)) [] -- (2 + 3) + 4

[9]

```haskell
evalL (Add (Add (Val 2) (Val 3)) (Val 4)) s0

        evalL (Add (Val 2) (Val 3)) s0 
                evalL (Val 2) s0     ~~>   2:s0
                evalL (Val 3) (2:s0) ~~> 3:2:s0
        ~~>   5:s0
        
        evalL (Val 4)
        ~~> 4:5:s0

~~>
9:s0
```

----
###### 연습문제
연산자의 오른쪽부터 먼저 계산하는 `evalR` 을 정의하라.

In [8]:
-- evalR :: Expr -> Stack -> Stack

----

In [9]:
-- Kont는 스택 변환 함수의 타입을 줄여서 쓴 것
type Kont = Stack -> Stack

pushK :: Int -> Kont
pushK n = push n

addLK :: Kont
addLK = addL

evalL' :: Expr -> Kont
evalL' (Val n)     = pushK n
evalL' (Add e1 e2) = addLK . evalL' e2 . evalL' e1

결국 evalL과 같은 일을 하는 함수인데 이렇게 포장을 좀 바꾸니
evalL'는 이전 단계 스택을 받아 다음 단계의 스택을 내놓는 함수로 이해할 수 있다.
즉 스택을 활용한 인터프리터는 스택 변환 함수를 만들어내는 함수인 것이다.
   
계산 결과는 `evalL`과 당연히 같다.

In [10]:
evalL' (Add (Add (Val 2) (Val 3)) (Val 4)) [] -- (2 + 3) + 4

[9]

하지만 이렇게 정의를 스택 변환 함수를 만들어내는 함수로써 인터프리터를 이해할 수 있으므로
합성함수의 성질을 이용한 아래와 같은 식 전개가 가능하다.

```haskell
  evalL' (Add (Add (Val 2) (Val 3)) (Val 4)) s0
= addLK . evalL' (Add (Val 2) (Val 3)) . eval (Val 4)  $ s0
= addLK . (addLK . pushK 2 . pushK 3) . pushK 4  $ s0
= addLK .  addLK . pushK 2 . pushK 3  . pushK 4  $ s0
= addLK (addLK (pushK 2 (pushK 3 (pushK 4 s0))))
= addLK (addLK (pushK 2 (pushK 3 (4 : s0))))
= addLK (addLK (pushK 2 (3 : 4 : s0)))
= addLK (addLK (2 : 3 : 4 : s0))
= addLK (5 : 4 : s0)
= addLK (9 : s0)
```

----
###### 연습문제
컨티뉴에이션 즉 "다음에 할 일" 개념을 바탕으로
연산자의 오른쪽부터 먼저 계산하는 `evalR'` 을 정의하라.
오른쪽부터 먼저 계산된 경우의 스택에 대해 "다음에 할 일"로
덧셈을 하는 `addRK :: Kont`를 정의하고 나면 그 다음부터는 쉽다.

In [11]:
-- addRK :: Kont

-- evalR' :: Expr -> Kont

----